In [1]:
import requests
import pandas as pd
import numpy as np
import json
from census import Census
from us import states
import os
import re
import time
import math

# Requesting Data from Open Data Philly

The property characteristic and assessment history data in this analysis is sourced from [OpenDataPhilly]([https://opendataphilly.org/datasets/philadelphia-properties-and-assessment-history/), an online repository providing access to data about the Philadelphia region. The data is maintained by the City of Philadelhia's the Office of Property Assessment and contains records for all properties in Philadelphia.

The Property and Property Assessment datasets published to Carto and available to query via [SQL API](https://phl.carto.com/api/v2/sql) using PostgreSQL.
- **Property table**: opa_properties_public
- **Property Assessment table**: assessments

In [2]:
def requestOpenDataPhilly(query: str):
    '''
    Sends request to Open Data Philly's SQL API, and returns the results as a Pandas dataframe

    Args:
        query (str): query written in PostgreSQL

    Return:
        dataframe containing the results of the query
    '''
    print('Sending request to Open Data Philly API.')
    url = 'https://phl.carto.com/api/v2/sql'
    params = {'q':query}

    # send request
    response = requests.get(url, params=params)
    print(f"Open Data API Status Code: {response.status_code}")
    
    # sleep
    time.sleep(2)
    
    try:
        # convert response to df
        response_data = json.loads(str(response.text))                          
        data = response_data['rows']        
        print(f'Returning df for query: {query}.\n')                                    
        return pd.DataFrame.from_dict(data) 
    except:
        return pd.DataFrame() 




In [3]:
# establish queries
prop_query = "SELECT * FROM opa_properties_public" 
assess_2025 = "SELECT * FROM assessments WHERE year = '2025'"
assess_2024 = "SELECT * FROM assessments WHERE year = '2024'"
assess_2023 = "SELECT * FROM assessments WHERE year = '2023'"
assess_2022 = "SELECT * FROM assessments WHERE year = '2022'"
assess_2021 = "SELECT * FROM assessments WHERE year = '2021'"
assess_2020 = "SELECT * FROM assessments WHERE year = '2020'"
assess_2019 = "SELECT * FROM assessments WHERE year = '2019'"
assess_2018 = "SELECT * FROM assessments WHERE year = '2018'"
assess_2017 = "SELECT * FROM assessments WHERE year = '2017'"
assess_2016 = "SELECT * FROM assessments WHERE year = '2016'"
assess_2015 = "SELECT * FROM assessments WHERE year = '2015'"

In [4]:
# # get requests
# prop = requestOpenDataPhilly(prop_query)
# assess_2025 = requestOpenDataPhilly(assess_2025)
# assess_2024 = requestOpenDataPhilly(assess_2024)
# assess_2023 = requestOpenDataPhilly(assess_2023)
# assess_2022 = requestOpenDataPhilly(assess_2022)
# assess_2021 = requestOpenDataPhilly(assess_2021)
# assess_2020 = requestOpenDataPhilly(assess_2020)
# assess_2019 = requestOpenDataPhilly(assess_2019) 
# assess_2018 = requestOpenDataPhilly(assess_2018)  
# assess_2017 = requestOpenDataPhilly(assess_2017)
# assess_2016 = requestOpenDataPhilly(assess_2016)
# assess_2015 = requestOpenDataPhilly(assess_2015)

In [5]:
# # export files
# prop.to_csv("Data/properties.csv", index=False)
# assess_2025.to_csv("Data/assess_2025.csv", index=False)
# assess_2024.to_csv("Data/assess_2024.csv", index=False)
# assess_2023.to_csv("Data/assess_2023.csv", index=False)
# assess_2022.to_csv("Data/assess_2022.csv", index=False)
# assess_2021.to_csv("Data/assess_2021.csv", index=False)
# assess_2020.to_csv("Data/assess_2020.csv", index=False)
# assess_2019.to_csv("Data/assess_2019.csv", index=False)
# assess_2018.to_csv("Data/assess_2018.csv", index=False)
# assess_2017.to_csv("Data/assess_2017.csv", index=False)
# assess_2016.to_csv("Data/assess_2016.csv", index=False)
# assess_2015.to_csv("Data/assess_2015.csv", index=False)


In [6]:
# read csvs
prop = pd.read_csv("Data/properties.csv")
assess_2025 = pd.read_csv("Data/assess_2025.csv")
assess_2024 = pd.read_csv("Data/assess_2024.csv")
assess_2023 = pd.read_csv("Data/assess_2023.csv")
assess_2022 = pd.read_csv("Data/assess_2022.csv")
assess_2021 = pd.read_csv("Data/assess_2021.csv")
assess_2020 = pd.read_csv("Data/assess_2020.csv")
assess_2019 = pd.read_csv("Data/assess_2019.csv")
assess_2018 = pd.read_csv("Data/assess_2018.csv")
assess_2017 = pd.read_csv("Data/assess_2017.csv")
assess_2016 = pd.read_csv("Data/assess_2016.csv")
assess_2015 = pd.read_csv("Data/assess_2015.csv")

C:\Users\julia\AppData\Local\Temp\ipykernel_64272\2771757085.py:2: DtypeWarning: Columns (6,18,27,29,78) have mixed types. Specify dtype option on import or set low_memory=False.
  prop = pd.read_csv("Data/properties.csv")


## Reading Real Estate Delinquency Data

Real Estate Delinquency data is available on Open Data Philly aggregated by location. To access this data at the parcel level, email Roman Stratkovsky <roman.stratkovsky@phila.gov>, Chief Data Intelligence Officer at the City of Philadelphia's Department of Revenue.

In [7]:
delinq = pd.read_csv("Data/real_estate_delinquency.csv")

## Check the shape of each dataframe

In [8]:
# property df size
prop.shape

(584103, 81)

In [9]:
# assessment df size
for df in [assess_2025, assess_2024, assess_2023, assess_2022, assess_2021, assess_2020, assess_2019, assess_2018, assess_2017, assess_2016, assess_2015]:
    print(df.shape)

(584014, 10)
(582676, 10)
(581527, 10)
(579221, 10)
(578012, 10)
(576751, 10)
(574880, 10)
(573566, 10)
(573539, 10)
(573552, 10)
(573545, 10)


In [10]:
# real delinquency data
delinq.shape

(1400509, 7)

The property characteristics dataframe has 81 columns, many of which are of no relevance to Real Estate Delinquency. Therefore, a subset of the columns was selected based on institutional knowledge and relevance to the research question.

In [11]:
# create copy of properties df
prop_clean = prop.loc[:,['parcel_number', 'category_code_description','exterior_condition','interior_condition','homestead_exemption','mailing_zip','number_of_bathrooms'
                         ,'number_of_bedrooms','number_stories','sale_date','sale_price','total_area','total_livable_area','unit','year_built','zip_code']].copy(deep=True)

### Concat and join Assessment dataframe

Assessment dataframes contain the property assessment for each year from 2015 to 2025. Join the individual dataframes so we can have a 1-year, 3-year, and 5-year look back for the years 2025-2020.

In [47]:
# create copies of the asessments df
assess_2025_clean = assess_2025.loc[:,['parcel_number','year','market_value','taxable_land','taxable_building','exempt_land','exempt_building']].copy(deep=True)
assess_2024_clean = assess_2024.loc[:,['parcel_number','year','market_value','taxable_land','taxable_building','exempt_land','exempt_building']].copy(deep=True)
assess_2023_clean = assess_2023.loc[:,['parcel_number','year','market_value','taxable_land','taxable_building','exempt_land','exempt_building']].copy(deep=True)
assess_2022_clean = assess_2022.loc[:,['parcel_number','year','market_value','taxable_land','taxable_building','exempt_land','exempt_building']].copy(deep=True)
assess_2021_clean = assess_2021.loc[:,['parcel_number','year','market_value','taxable_land','taxable_building','exempt_land','exempt_building']].copy(deep=True)
assess_2020_clean = assess_2020.loc[:,['parcel_number','year','market_value','taxable_land','taxable_building','exempt_land','exempt_building']].copy(deep=True)
assess_2019_clean = assess_2019.loc[:,['parcel_number','year','market_value','taxable_land','taxable_building','exempt_land','exempt_building']].copy(deep=True)
assess_2018_clean = assess_2018.loc[:,['parcel_number','year','market_value','taxable_land','taxable_building','exempt_land','exempt_building']].copy(deep=True)
assess_2017_clean = assess_2017.loc[:,['parcel_number','year','market_value','taxable_land','taxable_building','exempt_land','exempt_building']].copy(deep=True)
assess_2016_clean = assess_2016.loc[:,['parcel_number','year','market_value','taxable_land','taxable_building','exempt_land','exempt_building']].copy(deep=True)
assess_2015_clean = assess_2015.loc[:,['parcel_number','year','market_value','taxable_land','taxable_building','exempt_land','exempt_building']].copy(deep=True)

In [48]:
# concat the assessment years together
assess_all = pd.concat([assess_2025_clean, assess_2024_clean, assess_2023_clean, assess_2022_clean, assess_2021_clean, assess_2020_clean, 
                        assess_2019_clean,assess_2018_clean, assess_2017_clean, assess_2016_clean, assess_2015_clean], axis=0, ignore_index=True)

# create taxable and exempt fields
assess_all['taxable'] = assess_all['taxable_land'] + assess_all['taxable_building']
assess_all['exempt'] = assess_all['exempt_land'] + assess_all['exempt_building']

# create fields for the 1, 3, and 5 year joins
assess_all['1yr'] = assess_all['year'] - 1
assess_all['3yr'] = assess_all['year'] - 3
assess_all['5yr'] = assess_all['year'] - 5

# select a subset of columns for each df
assess_1yr = assess_all.loc[:,['parcel_number','year','taxable','exempt']].copy(deep=True)
assess_3yr = assess_all.loc[:,['parcel_number','year','taxable','exempt']].copy(deep=True)
assess_5yr = assess_all.loc[:,['parcel_number','year','taxable','exempt']].copy(deep=True)

# rename columns to indicate the year of assessment
assess_1yr.rename(columns={'taxable':'taxable_1yr','exempt':'exempt_1yr'}, inplace=True)
assess_3yr.rename(columns={'taxable':'taxable_3yr','exempt':'exempt_3yr'}, inplace=True)
assess_5yr.rename(columns={'taxable':'taxable_5yr','exempt':'exempt_5yr'}, inplace=True)

In [49]:

# join to last year's assessments
assess_clean = pd.merge(assess_all, assess_1yr, how='inner', left_on=['parcel_number','1yr'], right_on=['parcel_number','year'])
assess_clean.drop(columns=['year_y'], inplace=True)
assess_clean.rename(columns={'year_x':'year'}, inplace=True)

# join to the assessments from 3 years ago
assess_clean = pd.merge(assess_clean, assess_3yr, how='inner', left_on=['parcel_number','3yr'], right_on=['parcel_number','year'])
assess_clean.drop(columns=['year_y'], inplace=True)
assess_clean.rename(columns={'year_x':'year'}, inplace=True)

# join to the assessments from 5 years ago
assess_clean = pd.merge(assess_clean, assess_5yr, how='inner', left_on=['parcel_number','5yr'], right_on=['parcel_number','year'])
assess_clean.drop(columns=['year_y'], inplace=True)
assess_clean.rename(columns={'year_x':'year'}, inplace=True)

# drop the additional year columns
assess_clean.drop(columns=['1yr','3yr','5yr'], inplace=True)

In [50]:
assess_clean.shape

(3445780, 15)

## Pivot Delinquent dataframe

Pivot table so there is one record per property instead of one record per property and year.

In [63]:
# create copy of the delinquency df
delinq_copy = delinq.copy(deep=True)

In [72]:
# rename columns
delinq_copy.rename(columns={'snapshot_year':'year',
                            'principal_balance':'principal',
                            'interest_balance':'interest',
                            'penalty_balance':'penalty',
                            'other_balance':'other',
                            'total_balance':'total'},
                            inplace=True)

In [74]:
# drop years before 2019
delinq_copy = delinq_copy[delinq_copy.year >= 2019]

In [76]:
# pivot table
delinq_clean = delinq_copy.pivot(
    index = 'parcel_number',
    columns = 'year',
    values = ['principal','interest','penalty','other','total']
)

# flatten multiindex
delinq_clean.columns = [f'{col[0]}_{col[1]}' for col in delinq_clean.columns]

# reset index
delinq_clean = delinq_clean.reset_index()

In [78]:
delinq_clean.shape

(141840, 31)

## Check and convert data types

In [16]:
prop_clean.dtypes

parcel_number                  int64
category_code_description     object
exterior_condition            object
interior_condition            object
homestead_exemption            int64
mailing_zip                   object
number_of_bathrooms          float64
number_of_bedrooms           float64
number_stories               float64
sale_date                     object
sale_price                   float64
total_area                   float64
total_livable_area           float64
unit                          object
year_built                   float64
zip_code                     float64
dtype: object

Exterior condition is a numerical scale from 1 to 9 where 1 indicates newer construction, 9 indicates properties that are structurally compromised, and 0 indicates that exterior condition is not applicable. Remove errornous values, change 0s to NAs, and convert to float type.

In [17]:
# check unique values for exterior condition
prop_clean.exterior_condition.unique()

# 6 records where exterior condition is an invalid value
prop_clean[prop_clean.exterior_condition.isin(['P','E','A'])].head(10)

# change invalid records to NA
prop_clean.loc[(prop_clean.exterior_condition.isin(['P','E','A'])), 'exterior_condition'] = np.nan

# convert exterior condition to float
prop_clean.exterior_condition = prop_clean.exterior_condition.astype(float)

# change 0 (not applicable) records to NA
prop_clean.loc[(prop_clean.exterior_condition == 0), 'exterior_condition'] = np.nan

# confirm unqiue values of exterior condition
prop_clean.exterior_condition.unique()

array([nan,  4.,  1.,  3.,  5.,  7.,  2.,  6.,  8.])

Interior condition is a numerical scale from 1 to 8 where 1 indicates newer construction, 7 indicates properties that are structurally compromised, and 0 indicates that interior condition is not applicable. Remove errornous values, change 0s to NAs, and convert to float type.

In [18]:
# check unique values for interior condition
prop_clean.interior_condition.unique()

# 6 records where exterior condition is an invalid value
prop_clean[prop_clean.interior_condition.isin(['P','E','A'])].head(10)

# change invalid records to NA
prop_clean.loc[(prop_clean.interior_condition.isin(['P','E','A'])), 'interior_condition'] = np.nan

# convert interior condition to float
prop_clean.interior_condition = prop_clean.interior_condition.astype(float)

# change 0 (not applicable) records to NA
prop_clean.loc[(prop_clean.interior_condition == 0), 'interior_condition'] = np.nan

# confirm unqiue values of exterior condition
prop_clean.interior_condition.unique()

array([nan,  4.,  1.,  2.,  3.,  5.,  6.,  7.,  8.])

Unit, if exists, is a condo unit number. We care whether of not a property is a condo so we will convert unit to a boolean.

In [19]:
prop_clean.unit = prop_clean.unit.apply(lambda x: 0 if x == None else 1)

In [20]:
# convert from float to int for count data
prop_clean.number_of_bathrooms = prop_clean.number_of_bathrooms.astype('Int64')
prop_clean.number_of_bedrooms = prop_clean.number_of_bedrooms.astype('Int64')
prop_clean.number_stories = prop_clean.number_stories.astype('Int64')

# convert year built from float to string and remove floating point values
prop_clean.year_built = prop_clean.year_built.apply(lambda x: np.nan if pd.isna(x) else str(x)[:-2])

# convert zip code from float to string and remove floating point values
prop_clean.zip_code = prop_clean.zip_code.apply(lambda x: np.nan if pd.isna(x) else str(x)[:-2])

# convert from object to date
prop_clean.sale_date = pd.to_datetime(prop_clean.sale_date, format='%Y-%m-%dT%H:%M:%SZ', errors='coerce')

In [21]:
prop_clean.dtypes

parcel_number                         int64
category_code_description            object
exterior_condition                  float64
interior_condition                  float64
homestead_exemption                   int64
mailing_zip                          object
number_of_bathrooms                   Int64
number_of_bedrooms                    Int64
number_stories                        Int64
sale_date                    datetime64[ns]
sale_price                          float64
total_area                          float64
total_livable_area                  float64
unit                                  int64
year_built                           object
zip_code                             object
dtype: object

In [22]:
assess_2025_clean.dtypes

parcel_number         int64
year                  int64
market_value          int64
taxable_land        float64
taxable_building    float64
exempt_land         float64
exempt_building     float64
dtype: object

In [23]:
delinq_clean.dtypes

parcel_number          int64
snapshot_year          int64
principal_balance    float64
interest_balance     float64
penalty_balance      float64
other_balance        float64
total_balance        float64
dtype: object

## Check for missing values in the Property dataset

The property characteristics dataset is rife with missing values due to the method of data collection. This dataset is supported and maintained by the City of Philadelphia’s Office of Property Assessment (OPA). According to the OPA Mass Appraisal [Methodology Documentation](https://www.phila.gov/media/20240920162052/opa-tax-year-2025-mass-appraisal-valuation-methodology.pdf), data on individual properties is collected by OPA evaluators either in-person or virtually. In-person data collections consist of physically visiting a property and collecting observations from the curb. OPA evaluators may only conduct an internal inspection if they are invited in by an occupant. Virtual inspections are conducted by utilizing aerial photography or online listings. Therefore, data is limited by circumstance and online availability.

In [24]:
# count missing values
prop_clean.isna().sum()

parcel_number                    0
category_code_description       17
exterior_condition           43666
interior_condition           43689
homestead_exemption              0
mailing_zip                   2513
number_of_bathrooms          86243
number_of_bedrooms           81463
number_stories               72963
sale_date                     2725
sale_price                    2750
total_area                     726
total_livable_area           43203
unit                             0
year_built                   43200
zip_code                       146
dtype: int64

## Many missing values are coming from vacant properties

There are around 42,000 missing values for exterior condition, interior condition, number of bathrooms, number of bedrooms, number of stories, total livable area, and year built where the property is vacant. Vacant properties don't have bathrooms, bedrooms, stories, livable areas, or a year built since there is no structure. Therefore, we must correct these values.

In [25]:
prop_clean[prop_clean.category_code_description.isin(['VACANT LAND','VACANT LAND - RESIDENTIAL','VACANT LAND - NON-RESIDENTIAL'])].isna().sum()

parcel_number                    0
category_code_description        0
exterior_condition           42862
interior_condition           42865
homestead_exemption              0
mailing_zip                     96
number_of_bathrooms          42828
number_of_bedrooms           42824
number_stories               42850
sale_date                      760
sale_price                     765
total_area                     193
total_livable_area           42745
unit                             0
year_built                   42745
zip_code                       106
dtype: int64

## Fix records for vacant properties

In [26]:
# change the bathrooms from NA to 0 if the property is vacant
prop_clean.loc[(prop_clean.category_code_description.isin(['VACANT LAND','VACANT LAND - RESIDENTIAL','VACANT LAND - NON-RESIDENTIAL'])) & (prop_clean.number_of_bathrooms.isna()), 'number_of_bathrooms'] = 0

# change the bedrooms from NA to 0 if the property is vacant
prop_clean.loc[(prop_clean.category_code_description.isin(['VACANT LAND','VACANT LAND - RESIDENTIAL','VACANT LAND - NON-RESIDENTIAL'])) & (prop_clean.number_of_bedrooms.isna()), 'number_of_bedrooms'] = 0

# change the stories from NA to 0 if the property is vacant
prop_clean.loc[(prop_clean.category_code_description.isin(['VACANT LAND','VACANT LAND - RESIDENTIAL','VACANT LAND - NON-RESIDENTIAL'])) & (prop_clean.number_stories.isna()), 'number_stories'] = 0

# change the total livable area from NA to 0 if the property is vacant
prop_clean.loc[(prop_clean.category_code_description.isin(['VACANT LAND','VACANT LAND - RESIDENTIAL','VACANT LAND - NON-RESIDENTIAL'])) & (prop_clean.total_livable_area.isna()), 'total_livable_area'] = 0

# change year built to 2029
prop_clean.loc[(prop_clean.category_code_description.isin(['VACANT LAND','VACANT LAND - RESIDENTIAL','VACANT LAND - NON-RESIDENTIAL'])) & (prop_clean.year_built.isna()), 'year_built'] = 2029


Exterior conditions is a numerical scale from 1 to 9 where 1 indicates newer construction and 9 indictates properties that are structurally compromised. Therefore, we will change NA to 10 for vacant properties. 

In [27]:
# change the exterior condition from NA to 10 if the property is vacant
prop_clean.loc[(prop_clean.category_code_description.isin(['VACANT LAND','VACANT LAND - RESIDENTIAL','VACANT LAND - NON-RESIDENTIAL'])) & (prop_clean.exterior_condition.isna()), 'exterior_condition'] = 10

Interior condition is a numerical scale from 1 to 7 where 1 is new and or newly rehabed and 7 is structurally compromised. Therefore, we will change NA to 9 for vacant properties.

In [28]:
# change the interior condition from NA to 8 if the property is vacant
prop_clean.loc[(prop_clean.category_code_description.isin(['VACANT LAND','VACANT LAND - RESIDENTIAL','VACANT LAND - NON-RESIDENTIAL'])) & (prop_clean.interior_condition.isna()), 'interior_condition'] = 9

In [29]:
prop_clean[prop_clean.category_code_description.isin(['VACANT LAND','VACANT LAND - RESIDENTIAL','VACANT LAND - NON-RESIDENTIAL'])].isna().sum()

parcel_number                  0
category_code_description      0
exterior_condition             0
interior_condition             0
homestead_exemption            0
mailing_zip                   96
number_of_bathrooms            0
number_of_bedrooms             0
number_stories                 0
sale_date                    760
sale_price                   765
total_area                   193
total_livable_area             0
unit                           0
year_built                     0
zip_code                     106
dtype: int64

There still exists many records with the number of bedrooms, bathrooms, and stories missing. 

In [30]:
prop_clean.isna().sum()

parcel_number                    0
category_code_description       17
exterior_condition             804
interior_condition             824
homestead_exemption              0
mailing_zip                   2513
number_of_bathrooms          43415
number_of_bedrooms           38639
number_stories               30113
sale_date                     2725
sale_price                    2750
total_area                     726
total_livable_area             458
unit                             0
year_built                     455
zip_code                       146
dtype: int64

As discussed earlier, missing values for number of bedrooms, bathrooms, and stories in the OPA property dataset are a result of circumstance and online availability. The unavailability of these variables seems to follow a pattern with the category code description. Single family and multifamily homes are more likely to have counts of bedrooms and bathrooms than other non-residential building types. Simiarly, commercial, industrial, and mixed use properties tend to have missing values for number of stories. This is most likely because online real estate marketplaces like Zillow advertise or estimate the number of bedrooms, bathrooms, and bathrooms for residential properties. Therefore, we suspect that the missingness of these variables is random, but not completely random. 

In [31]:
# counts by category code description
prop_clean[['parcel_number','number_of_bedrooms','number_of_bathrooms','number_stories']].groupby(prop_clean.category_code_description).agg(['count']) 

,parcel_number,number_of_bedrooms,number_of_bathrooms,number_stories
,count,count,count,count
category_code_description,,,,
APARTMENTS > 4 UNITS,3476,96,85,3235
COMMERCIAL,12665,29,25,303
GARAGE - COMMERCIAL,108,25,27,63
GARAGE - RESIDENTIAL,584,514,514,547
HOTEL,127,1,0,110
INDUSTRIAL,4082,4,4,589
MIXED USE,14274,206,187,1152
MULTI FAMILY,41141,38354,37735,40755


In [32]:
# count, mean, and mode for number of bedrooms by building category
prop_clean[['number_of_bedrooms']].groupby(prop_clean.category_code_description).agg(['count','mean', 'median', pd.Series.mode]) 

number_of_bedrooms                         
                                           count      mean median    mode
category_code_description                                                
APARTMENTS  > 4 UNITS                         96    3.9375    3.0       0
COMMERCIAL                                    29  0.827586    0.0       0
GARAGE - COMMERCIAL                           25      0.64    0.0       0
GARAGE - RESIDENTIAL                         514  0.031128    0.0       0
HOTEL                                          1       8.0    8.0       8
INDUSTRIAL                                     4      0.75    0.0       0
MIXED USE                                    206  2.271845    3.0       0
MULTI FAMILY                               38354  1.857329    0.0       0
OFFICES                                        3       0.0    0.0       0
RETAIL                                         2       1.0    1.0  [0, 2]
SINGLE FAMILY                             463102   2.48916    3.0       3
SPECIAL PURPOSE                               11       1.0    0.0       0
VACANT LAND                                41894  0.006445    0.0       0
VACANT LAND - NON-RESIDENTIAL                131       0.0    0.0       0
VACANT LAND - RESIDENTIAL                   1090  0.022936    0.0       0

In [33]:
# count, mean, and mode for number of bathrooms by building category
prop_clean[['number_of_bathrooms']].groupby(prop_clean.category_code_description).agg(['count','mean', 'median', pd.Series.mode]) 

number_of_bathrooms                         
                                            count      mean median    mode
category_code_description                                                 
APARTMENTS  > 4 UNITS                          85  2.317647    0.0       0
COMMERCIAL                                     25      0.32    0.0       0
GARAGE - COMMERCIAL                            27  0.259259    0.0       0
GARAGE - RESIDENTIAL                          514  0.013619    0.0       0
HOTEL                                           0      <NA>   <NA>      []
INDUSTRIAL                                      4      0.25    0.0       0
MIXED USE                                     187  1.149733    1.0       0
MULTI FAMILY                                37735  1.001034    0.0       0
OFFICES                                         3       0.0    0.0       0
RETAIL                                          2       0.5    0.5  [0, 1]
SINGLE FAMILY                              458978  0.982903    1.0       1
SPECIAL PURPOSE                                11  0.454545    0.0       0
VACANT LAND                                 41894   0.00265    0.0       0
VACANT LAND - NON-RESIDENTIAL                 131       0.0    0.0       0
VACANT LAND - RESIDENTIAL                    1090  0.006422    0.0       0

In [34]:
# count, mean, and mode for number of stories by building category
prop_clean[['number_stories']].groupby(prop_clean.category_code_description).agg(['count','mean', 'median', pd.Series.mode]) 

number_stories                      
                                       count      mean median mode
category_code_description                                         
APARTMENTS  > 4 UNITS                   3235  3.868006    3.0    3
COMMERCIAL                               303   2.09901    2.0    1
GARAGE - COMMERCIAL                       63  2.761905    1.0    1
GARAGE - RESIDENTIAL                     547  1.023766    1.0    1
HOTEL                                    110  8.672727    5.0    4
INDUSTRIAL                               589  1.612903    1.0    1
MIXED USE                               1152  2.583333    3.0    3
MULTI FAMILY                           40755  2.375537    2.0    2
OFFICES                                  720  4.770833    3.0    1
RETAIL                                    12      1.75    1.5    1
SINGLE FAMILY                         463363  1.914288    2.0    2
SPECIAL PURPOSE                           18  2.111111    2.0    2
VACANT LAND                            41894  0.006254    0.0    0
VACANT LAND - NON-RESIDENTIAL            131       0.0    0.0    0
VACANT LAND - RESIDENTIAL               1090  0.113761    0.0    0

## Impute or drop missing values in the property dataset

We will impute the missing values of number of bedrooms, bathrooms, and stories with a stratified imputation. Based on subject matter knowledge, we will round the number of bedrooms down, the number of bathrooms up, and the number of stories to the closest digit to have interger counts.

In [35]:
prop_clean.number_of_bedrooms = prop_clean.groupby('category_code_description')['number_of_bedrooms'].transform(lambda x: x.fillna(value= math.floor(x.mean())))

In [36]:
prop_clean.number_of_bathrooms = prop_clean.groupby('category_code_description')['number_of_bathrooms'].transform(lambda x: x.fillna(math.ceil(x.mean())) if not pd.isna(x.mean()) else x)

In [37]:
prop_clean.number_stories = prop_clean.groupby('category_code_description')['number_stories'].transform(lambda x: x.fillna(value= round(x.mean())))

In [38]:
prop_clean.isna().sum()

parcel_number                   0
category_code_description      17
exterior_condition            804
interior_condition            824
homestead_exemption             0
mailing_zip                  2513
number_of_bathrooms           144
number_of_bedrooms             17
number_stories                 17
sale_date                    2725
sale_price                   2750
total_area                    726
total_livable_area            458
unit                            0
year_built                    455
zip_code                      146
dtype: int64

In [39]:
# count the number of records that would be dropped if any records with missing values were dropped
len(prop_clean[prop_clean.isna().any(axis = 'columns')])

6365

We will drop the rest of the records with missing values since missingness in the property dataset is a random or completely random and less then 5% of the records would be dropped.

In [40]:
# drop missing records
prop_clean.dropna(inplace=True)

In [45]:
# check missing values
prop_clean.isna().sum()

parcel_number                0
category_code_description    0
exterior_condition           0
interior_condition           0
homestead_exemption          0
mailing_zip                  0
number_of_bathrooms          0
number_of_bedrooms           0
number_stories               0
sale_date                    0
sale_price                   0
total_area                   0
total_livable_area           0
unit                         0
year_built                   0
zip_code                     0
dtype: int64

## Check for missing values in the property assessment and Real Estate Delinquency datasets

In [57]:
assess_clean.isna().sum()

parcel_number       0
year                0
market_value        0
taxable_land        1
taxable_building    3
exempt_land         0
exempt_building     0
taxable             4
exempt              0
taxable_1yr         5
exempt_1yr          0
taxable_3yr         2
exempt_3yr          0
taxable_5yr         1
exempt_5yr          0
dtype: int64

In [58]:
assess_clean[assess_clean.taxable_building.isna()]

,parcel_number,year,market_value,taxable_land,taxable_building,exempt_land,exempt_building,taxable,exempt,taxable_1yr,exempt_1yr,taxable_3yr,exempt_3yr,taxable_5yr,exempt_5yr
1263613,191397500,2023,408000,89700.0,NaN,318300.0,0.0,NaN,318300.0,59800.0,0.0,59800.0,0.0,59800.0,0.0
1337472,302150100,2023,256000,61200.0,NaN,194800.0,0.0,NaN,194800.0,61200.0,194800.0,61200.0,194800.0,256000.0,0.0
1688473,885948280,2023,317500,317500.0,NaN,0.0,0.0,NaN,0.0,245300.0,0.0,204417.0,40883.0,0.0,108900.0


The final assessment table is the result of inner joins, so we have all properties that have an assessment for the past 5 years. Missing values here seem to be the result of input error so we will drop those records.

In [59]:
assess_clean.dropna(inplace=True)

In [39]:
def clean_assessments(df, year: int):
    '''
    Cleans the assessment dataframe by removing and renaming columns.

    Args:
        df (DataFrame): The dataframe to be cleaned.

    Returns:
        DataFrame: The cleaned dataframe.
    '''
    # Remove unnecessary columns
    df_clean = df.loc[:,['parcel_number','market_value','taxable_land','taxable_building','exempt_land','exempt_building']].copy(deep=True)
    
    # Rename columns for clarity
    df_clean.rename(columns={
        'market_value':'market_value_' + str(year),
        'taxable_land':'taxable_land_' + str(year),
        'taxable_building':'taxable_building_' + str(year),
        'exempt_land':'exempt_land_'  + str(year),
        'exempt_building':'exempt_building_'  + str(year)
    }, inplace=True)
    
    return df_clean


assess_2025_clean = clean_assessments(assess_2025, 2025)
assess_2024_clean = clean_assessments(assess_2024, 2024)    
assess_2023_clean = clean_assessments(assess_2023, 2023)
assess_2022_clean = clean_assessments(assess_2022, 2022)
assess_2021_clean = clean_assessments(assess_2021, 2021)
assess_2020_clean = clean_assessments(assess_2020, 2020)
assess_2019_clean = clean_assessments(assess_2019, 2019)

In [40]:
# merge assessment df
assess_clean = pd.merge(assess_2025_clean, assess_2024_clean, how='inner', on='parcel_number')
assess_clean = pd.merge(assess_clean, assess_2023_clean, how='inner', on='parcel_number')
assess_clean = pd.merge(assess_clean, assess_2022_clean, how='inner', on='parcel_number')
assess_clean = pd.merge(assess_clean, assess_2021_clean, how='inner', on='parcel_number')
assess_clean = pd.merge(assess_clean, assess_2020_clean, how='inner', on='parcel_number')
assess_clean = pd.merge(assess_clean, assess_2019_clean, how='inner', on='parcel_number')

In [ ]:
assess_clean['taxable_chg_1_yr'] = assess_clean['taxable_land_2025'] + assess_clean['taxable_building_2025'] - assess_clean['taxable_land_2024'] - assess_clean['taxable_land_2024']
assess_clean['taxable_chg_2_yr'] = assess_clean['taxable_land_2025'] + assess_clean['taxable_building_2025'] - assess_clean['taxable_land_2023'] - assess_clean['taxable_land_2023']
assess_clean['taxable_chg_3_yr'] = assess_clean['taxable_land_2025'] + assess_clean['taxable_building_2025'] - assess_clean['taxable_land_2022'] - assess_clean['taxable_land_2022']
assess_clean['taxable_chg_4_yr'] = assess_clean['taxable_land_2025'] + assess_clean['taxable_building_2025'] - assess_clean['taxable_land_2021'] - assess_clean['taxable_land_2021']
assess_clean['taxable_chg_5_yr'] = assess_clean['taxable_land_2025'] + assess_clean['taxable_building_2025'] - assess_clean['taxable_land_2020'] - assess_clean['taxable_land_2020']

(574865, 36)

In [9]:
# set max rows in pandas to 100 
pd.set_option('display.max_rows', 100)

https://www.phila.gov/media/20240920162052/opa-tax-year-2025-mass-appraisal-valuation-methodology.pdf

Inspections are limited to what can be seen from the curb. Interior inspections are done by owner
invitation. This process helps OPA evaluators to collect data on characteristics of properties throughout
the City. Other tools, such as permit information from the Department of Licenses + Inspections, aerial
and street level photography, and private property listings that describe other characteristics are also
used. 

parcel_number                     0
category_code_description        17 
exterior_condition            43556 -- impute mode
interior_condition            43570 -- impute mode
homestead_exemption               0 
mailing_zip                    2513 
number_of_bathrooms           86243 -- model
number_of_bedrooms            81463 -- model
number_stories                72963 -- model
sale_date                      2723 -- check to see if these are old properties
sale_price                     2750 -- ?
total_area                      726 -- 
total_livable_area            43203 -- model 
unit                         542962
year_built                    43200
zip_code                        146

In [10]:
prop_clean.isna().sum()

parcel_number                     0
category_code_description        17
exterior_condition            43556
interior_condition            43570
homestead_exemption               0
mailing_zip                    2513
number_of_bathrooms           86243
number_of_bedrooms            81463
number_stories                72963
sale_date                      2723
sale_price                     2750
total_area                      726
total_livable_area            43203
unit                         542962
year_built                    43200
zip_code                        146
dtype: int64

In [ ]:
# reformat the category_code_description column and create dummies
prop_clean.category_code_description = prop_clean.category_code_description.apply(lambda x: None if x == None else x.split('-')[0].strip().replace(' ', '_').lower()) 
prop_clean = pd.get_dummies(prop_clean, columns=['category_code_description'], prefix= '', prefix_sep='')

# create indicator for homestead exemption
prop_clean['homestead_exemption'] = prop_clean.homestead_exemption.apply(lambda x: 0 if x < 1 else 1)

# create indicator for mailing zip outside of the city limits
prop_clean['mailing_outside_city'] = prop_clean.mailing_zip.apply(lambda x: 0 if x == None else (0 if x[:3] == '191' else 1))

# view total libable area by number of bedrooms
prop_clean.total_livable_area.groupby(prop_clean['number_of_bedrooms']).describe() ## bedrooms over 30 are incorrect
prop_clean.total_livable_area.groupby(prop_clean['number_of_bathrooms']).describe() ## bathrooms over 20 are incorrect

# indicator for whether a property has a condo unit
prop_clean['condo'] = prop_clean['unit'].apply(lambda x: 0 if x == None else 1)

In [ ]:
prop_clean.columns

Index(['cartodb_id', 'the_geom', 'the_geom_webmercator', 'assessment_date',
       'basements', 'beginning_point', 'book_and_page', 'building_code',
       'building_code_description', 'category_code', 'census_tract',
       'central_air', 'cross_reference', 'date_exterior_condition', 'depth',
       'exempt_building', 'exempt_land', 'exterior_condition', 'fireplaces',
       'frontage', 'fuel', 'garage_spaces', 'garage_type',
       'general_construction', 'geographic_ward', 'homestead_exemption',
       'house_extension', 'house_number', 'interior_condition', 'location',
       'mailing_address_1', 'mailing_address_2', 'mailing_care_of',
       'mailing_city_state', 'mailing_street', 'mailing_zip', 'market_value',
       'market_value_date', 'number_of_bathrooms', 'number_of_bedrooms',
       'number_of_rooms', 'number_stories', 'off_street_open',
       'other_building', 'owner_1', 'owner_2', 'parcel_number', 'parcel_shape',
       'quality_grade', 'recording_date', 'registry_number

In [ ]:
# check columns and data types
prop.dtypes

['parcel_number', 'category_code_description','homestead_exemption','mailing_zip','number_of_bathrooms','number_of_bedrooms',
 'number_stories','sale_date','sale_price','total_area','total_livable_area','unit','year_built','zip_code']

cartodb_id                         int64
the_geom                          object
the_geom_webmercator              object
assessment_date                   object
basements                         object
beginning_point                   object
book_and_page                     object
building_code                     object
building_code_description         object
category_code                     object
category_code_description         object
census_tract                      object
central_air                       object
cross_reference                   object
date_exterior_condition           object
depth                            float64
exempt_building                  float64
exempt_land                      float64
exterior_condition                object
fireplaces                       float64
frontage                         float64
fuel                              object
garage_spaces                    float64
garage_type                       object
general_construc

In [ ]:
# check column and data types
assess.dtypes

cartodb_id                int64
the_geom                 object
the_geom_webmercator     object
parcel_number            object
year                      int64
market_value              int64
taxable_land            float64
taxable_building        float64
exempt_land             float64
exempt_building         float64
dtype: object